In [1]:
%pip install transformers
%pip install tensorflow
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 4.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.6/11.1 MB 3.7 MB/s eta 0:00:03
   ------- -------------------------------- 2.1/11.1 MB 4.3 MB/s eta 0:00:03
   ------------- -------------------------- 3.7/11.1 MB 4.5 MB/s eta 0:00:02
   ------------------- -------------------- 5.5/11.1 MB 5.6 MB/s eta 0:00:01
   --------------------------- ------------ 7.6/11.1 MB 6.1 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.1 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.1 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 6.7 MB/s eta 0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl

In [1]:
# 필수 라이브러리 임포트
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 재현 가능한 결과를 위한 시드 설정
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

ModuleNotFoundError: No module named 'numpy'

In [ ]:
print(transformers.__version__)
print(tf.__version__)

4.44.2
2.17.0


In [ ]:
# pandas read_excel 함수 사용을 위한 openpyxl 설치
%pip install openpyxl -q

In [ ]:
# 하이퍼파라미터 및 설정 값들을 변수로 정의
TRAIN_DATA_URL = 'https://github.com/gzone2000/TEMP_TEST/raw/master/A_comment_train.xlsx'
TEST_DATA_URL = 'https://github.com/gzone2000/TEMP_TEST/raw/master/A_comment_test.xlsx'
BERT_MODEL_NAME = 'klue/bert-base'
MAX_LENGTH = 128  # BERT 입력 시퀀스 최대 길이
BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 1e-5
TEST_SIZE = 0.2

# 데이터 로드
try:
    comment_train = pd.read_excel(TRAIN_DATA_URL, engine='openpyxl')
    comment_test = pd.read_excel(TEST_DATA_URL, engine='openpyxl')
    print(f"훈련 데이터 크기: {comment_train.shape}")
    print(f"테스트 데이터 크기: {comment_test.shape}")
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {e}")
    raise

In [ ]:
comment_train.head()

,Unnamed: 0,data,label
0,0,재미는 있는데 시간이 짧은게 아쉽네요~,긍정
1,1,"OO 관련 내용은 우리 직원과는 거리가 멀었음, 특히, 사내에 홍보할 내용은 아니라고 봄",부정
2,2,스토리가 너무 딱딱해서 별로였음,부정
3,3,프로그램A 화이팅하세요!!,긍정
4,4,높은 곳에 올라가는 모습이 너무 위험해 보여요.,부정


In [ ]:
comment_test.count()

,0
Unnamed: 0,101
data,101
label,101


In [ ]:
# 데이터 전처리 및 정리
def preprocess_data(df):
    """데이터 전처리 함수"""
    # 불필요한 컬럼 제거
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis=1)
    
    # 결측값 확인 및 처리
    null_count = df.isnull().sum().sum()
    if null_count > 0:
        print(f"결측값 {null_count}개 발견되었습니다:")
        print(df.isnull().sum())
        df = df.dropna()  # 결측값이 있는 행 제거
    
    # 텍스트 데이터 기본 정리 (공백 제거 등)
    df['data'] = df['data'].str.strip()
    
    # 빈 텍스트 제거
    before_len = len(df)
    df = df[df['data'].str.len() > 0]
    after_len = len(df)
    if before_len != after_len:
        print(f"빈 텍스트 {before_len - after_len}개가 제거되었습니다.")
    
    return df.reset_index(drop=True)

# 훈련 데이터만 사용 (실제로는 train/validation split을 할 예정)
comment = preprocess_data(comment_train.copy())
print(f"전처리 후 데이터 크기: {comment.shape}")
print(f"레이블 분포:\n{comment['label'].value_counts()}")

In [ ]:
comment.head()

,Unnamed: 0,data,label
0,0,재미는 있는데 시간이 짧은게 아쉽네요~,긍정
1,1,"OO 관련 내용은 우리 직원과는 거리가 멀었음, 특히, 사내에 홍보할 내용은 아니라고 봄",부정
2,2,스토리가 너무 딱딱해서 별로였음,부정
3,3,프로그램A 화이팅하세요!!,긍정
4,4,높은 곳에 올라가는 모습이 너무 위험해 보여요.,부정


In [ ]:
comment.isnull().sum()

,0
Unnamed: 0,0
data,0
label,0


In [ ]:
# label 변환
#comment['label'] = comment['label'].replace(['부정', '긍정'],[0,1])
comment['label'] = comment['label'].apply(lambda x: 0 if x == '부정' else 1)

In [ ]:
comment.tail()

,Unnamed: 0,data,label
246,246,영상F서비스로 간편하게 설치!좋아요\n우리 회사화이팅!,1
247,247,모든 업무에서 맡은바 업무에 서 최선을 다하는 모습이 좋습니다! 화이팅 입니다.,1
248,248,"사내방송 특성상 최근 이슈화 되거나, 언급이 자주되는 '키워드'를 중심으로 뉴스를 ...",0
249,249,방송 시간이 너무 길어요.,0
250,250,"처음 들어보는 말들이 많은데,, 설명이 없어서 힘드네요.",0


In [ ]:
comment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  251 non-null    int64 
 1   data        251 non-null    object
 2   label       251 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.0+ KB


In [ ]:
# 데이터를 feature와 label로 분리
X = comment['data'].tolist()
y = comment['label'].tolist()

print(f"총 샘플 수: {len(X)}")
print(f"긍정 샘플: {sum(y)}개, 부정 샘플: {len(y) - sum(y)}개")

In [ ]:
# 훈련/검증 데이터 분할 (stratify를 사용하여 클래스 비율 유지)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    stratify=y, 
    test_size=TEST_SIZE, 
    random_state=RANDOM_SEED
)

print(f"훈련 데이터: {len(X_train)}개")
print(f"검증 데이터: {len(X_test)}개")
print(f"훈련 데이터 클래스 분포: 긍정 {sum(y_train)}개, 부정 {len(y_train) - sum(y_train)}개")
print(f"검증 데이터 클래스 분포: 긍정 {sum(y_test)}개, 부정 {len(y_test) - sum(y_test)}개")

In [ ]:
len(X_train), len(X_test), len(y_train), len(y_test)

(200, 51, 200, 51)

In [ ]:
X_train[:2]

['우리 회사 기업 이미지 활동에도 세심한 할동과 노력이 필요할때 같습니다',
 '간략한 브리핑이나 아님 끝나고 나서 오늘 전달한 콘텐츠에 대한 정리를 한번 해주시면 한 눈에 딱 들어와서 좋을 듯 합니다.']

In [ ]:
# BERT 모델 및 토크나이저 초기화
print(f"사용할 BERT 모델: {BERT_MODEL_NAME}")
print(f"최대 시퀀스 길이: {MAX_LENGTH}")

In [ ]:
# BERT 토크나이저와 모델 컴포넌트 임포트 및 초기화
from transformers import AutoConfig, BertTokenizerFast, TFBertForSequenceClassification

try:
    tokenizer = BertTokenizerFast.from_pretrained(BERT_MODEL_NAME)
    print(f"토크나이저 로드 완료. 어휘 크기: {tokenizer.vocab_size}")
except Exception as e:
    print(f"토크나이저 로드 중 오류 발생: {e}")
    raise

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenizer.vocab_size

32000

In [ ]:
#tokenizer.vocab

In [ ]:
# 텍스트를 BERT 입력 형태로 토크나이징
print("텍스트 토크나이징 중...")
train_encodings = tokenizer(
    X_train, 
    truncation=True, 
    padding=True, 
    max_length=MAX_LENGTH,
    return_tensors='tf'
)
test_encodings = tokenizer(
    X_test, 
    truncation=True, 
    padding=True, 
    max_length=MAX_LENGTH,
    return_tensors='tf'
)

print(f"훈련 데이터 토크나이징 완료. 형태: {train_encodings['input_ids'].shape}")
print(f"검증 데이터 토크나이징 완료. 형태: {test_encodings['input_ids'].shape}")

In [ ]:
print(train_encodings['input_ids'][0])

[2, 3616, 3769, 3646, 4661, 3746, 6509, 12541, 2470, 1892, 2328, 2145, 3973, 2052, 3677, 2085, 2775, 555, 2219, 3606, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(test_encodings['input_ids'][0])

[2, 4365, 2116, 16984, 2897, 4347, 2052, 2379, 3760, 22939, 2052, 2203, 2182, 18, 4243, 2116, 3760, 1521, 13486, 2182, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# TensorFlow 데이터셋 생성 및 최적화
print("TensorFlow 데이터셋 생성 중...")

# 훈련 데이터셋 생성 (셔플, 배치, 캐싱, 프리페치 적용)
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train))
train_dataset = train_dataset.shuffle(1000, seed=RANDOM_SEED).batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE)

# 검증 데이터셋 생성 (배치, 캐싱, 프리페치 적용)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test))
test_dataset = test_dataset.batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE)

print(f"데이터셋 생성 완료. 배치 크기: {BATCH_SIZE}")

In [ ]:
# BERT 모델 설정 확인
config = AutoConfig.from_pretrained(BERT_MODEL_NAME)
print(f"BERT 모델 설정:")
print(f"- 숨겨진 크기: {config.hidden_size}")
print(f"- 어텐션 헤드 수: {config.num_attention_heads}")
print(f"- 숨겨진 레이어 수: {config.num_hidden_layers}")
print(f"- 어휘 크기: {config.vocab_size}")
print(f"- 최대 위치 임베딩: {config.max_position_embeddings}")
config

BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.44.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

In [ ]:
# BERT 분류 모델 생성 및 컴파일
print("BERT 분류 모델 초기화 중...")

# 사전 훈련된 BERT 모델을 분류 작업에 맞게 초기화 (2개 클래스: 긍정/부정)
try:
    model = TFBertForSequenceClassification.from_pretrained(
        BERT_MODEL_NAME, 
        num_labels=2,  # 긍정/부정 2개 클래스
        from_pt=True   # PyTorch 모델에서 TensorFlow로 변환
    )
    print("BERT 모델 로드 완료")
except Exception as e:
    print(f"모델 로드 중 오류 발생: {e}")
    raise

# 옵티마이저, 손실 함수, 메트릭 설정
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

# 모델 컴파일
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

print(f"모델 컴파일 완료:")
print(f"- 학습률: {LEARNING_RATE}")
print(f"- 손실 함수: SparseCategoricalCrossentropy")
print(f"- 메트릭: {metrics}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 모델 훈련
print("모델 훈련 시작...")
print(f"훈련 에포크: {EPOCHS}")
print(f"배치 크기: {BATCH_SIZE}")

# 콜백 설정 (조기 종료, 체크포인트 저장 등)
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    )
]

# 모델 훈련 실행
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
    callbacks=callbacks,
    verbose=1
)

print("모델 훈련 완료!")

Epoch 1/10
13/13 [==============================] - 58s 591ms/step - loss: 0.5465 - accuracy: 0.8150 - val_loss: 0.4230 - val_accuracy: 0.9216
Epoch 2/10
13/13 [==============================] - 5s 371ms/step - loss: 0.2964 - accuracy: 0.9750 - val_loss: 0.2611 - val_accuracy: 0.9412
Epoch 3/10
13/13 [==============================] - 5s 357ms/step - loss: 0.1464 - accuracy: 0.9850 - val_loss: 0.1624 - val_accuracy: 0.9608
Epoch 4/10
13/13 [==============================] - 5s 357ms/step - loss: 0.0671 - accuracy: 1.0000 - val_loss: 0.1257 - val_accuracy: 0.9608
Epoch 5/10
13/13 [==============================] - 5s 384ms/step - loss: 0.0351 - accuracy: 1.0000 - val_loss: 0.1135 - val_accuracy: 0.9608
Epoch 6/10
13/13 [==============================] - 5s 358ms/step - loss: 0.0217 - accuracy: 1.0000 - val_loss: 0.1038 - val_accuracy: 0.9608
Epoch 7/10
13/13 [==============================] - 5s 375ms/step - loss: 0.0155 - accuracy: 1.0000 - val_loss: 0.1011 - val_accuracy: 0.9804
Epoch

In [ ]:
# 훈련 과정 시각화
plt.figure(figsize=(12, 4))

# 손실 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='훈련 손실')
plt.plot(history.history['val_loss'], label='검증 손실')
plt.title('모델 손실')
plt.xlabel('에포크')
plt.ylabel('손실')
plt.legend()

# 정확도 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='훈련 정확도')
plt.plot(history.history['val_accuracy'], label='검증 정확도')
plt.title('모델 정확도')
plt.xlabel('에포크')
plt.ylabel('정확도')
plt.legend()

plt.tight_layout()
plt.show()

# 최종 성능 출력
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print(f"\n최종 훈련 결과:")
print(f"- 훈련 정확도: {final_train_acc:.4f}")
print(f"- 검증 정확도: {final_val_acc:.4f}")
print(f"- 훈련 손실: {final_train_loss:.4f}")
print(f"- 검증 손실: {final_val_loss:.4f}")


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_113 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 110618882 (421.98 MB)
Trainable params: 110618882 (421.98 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 실제 테스트 데이터로 모델 성능 평가
print("실제 테스트 데이터로 모델 평가 중...")

# 레이블이 있는 테스트 데이터 사용 (전처리 적용)
comment_valid = preprocess_data(comment_test.copy())
print(f"평가용 테스트 데이터 크기: {comment_valid.shape}")
print(f"테스트 데이터 레이블 분포:\n{comment_valid['label'].value_counts()}")
comment_valid.head()

,Unnamed: 0,data
0,0,유익한 정보 감사합니다. 취미로 바다에 자주 나가는 편인데 이런 기술이 있는지 몰랐네요.
1,1,발빠른 영입 수고 했네요 고객들이 좋아합니다.
2,2,프로그램C 주제가 우리의 기술력이 바탕이 된다는점~~..항상 응원합니다
3,3,언제나 밝은 미소로 열심히 근무하시는 청원 경찰 분들께 감사드립니다. 항상 건강하시...
4,4,안전사고에 대한 개념도 함께 소개 되었으면 더 완벽했을 것 같네요
...,...,...
96,96,작년에 프로그램A를 재밋게 봤던 시청자로서 올해의 미니드라마도 매우 기대가 됩니다....
97,97,프로그램C 잘 보았습니다. 모든일의 바탕은 안전인것 같습니다. 모두를 보호하는 최고...
98,98,위험한 시설에 대한 설명도 부탁드립니다
99,99,구체적으로 어떤 활동을 해왔었고 앞으로 어떤활동을 할건지 잘 설명해줬으면 좋았을 것...


In [ ]:
comment_valid['data']

,data
0,유익한 정보 감사합니다. 취미로 바다에 자주 나가는 편인데 이런 기술이 있는지 몰랐네요.
1,발빠른 영입 수고 했네요 고객들이 좋아합니다.
2,프로그램C 주제가 우리의 기술력이 바탕이 된다는점~~..항상 응원합니다
3,언제나 밝은 미소로 열심히 근무하시는 청원 경찰 분들께 감사드립니다. 항상 건강하시...
4,안전사고에 대한 개념도 함께 소개 되었으면 더 완벽했을 것 같네요
...,...
96,작년에 프로그램A를 재밋게 봤던 시청자로서 올해의 미니드라마도 매우 기대가 됩니다....
97,프로그램C 잘 보았습니다. 모든일의 바탕은 안전인것 같습니다. 모두를 보호하는 최고...
98,위험한 시설에 대한 설명도 부탁드립니다
99,구체적으로 어떤 활동을 해왔었고 앞으로 어떤활동을 할건지 잘 설명해줬으면 좋았을 것...


In [ ]:
# 테스트 데이터 예측을 위한 전처리
print("테스트 데이터 토크나이징 및 예측 중...")

# 텍스트 데이터 추출
valid_texts = comment_valid['data'].tolist()
print(f"예측할 텍스트 수: {len(valid_texts)}")

# 토크나이징 (훈련 때와 동일한 설정 사용)
valid_encodings = tokenizer(
    valid_texts, 
    truncation=True, 
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors='tf'
)

# TensorFlow 데이터셋으로 변환
valid_dataset = tf.data.Dataset.from_tensor_slices(dict(valid_encodings))
valid_dataset = valid_dataset.batch(BATCH_SIZE)

# 예측 실행
predictions = model.predict(valid_dataset)
print("예측 완료!")

7/7 [==============================] - 3s 74ms/step


In [ ]:
predictions.logits

array([[-2.0256371 ,  2.8980608 ],
       [-2.4638686 ,  3.0682173 ],
       [-2.3047283 ,  3.1526093 ],
       [-2.421017  ,  2.9058363 ],
       [ 2.4947276 , -2.135936  ],
       [-2.39279   ,  3.0074892 ],
       [-2.3139982 ,  2.8844903 ],
       [ 2.7493455 , -2.296551  ],
       [ 2.530861  , -2.1216004 ],
       [ 2.564917  , -2.526381  ],
       [ 2.5758035 , -2.2003055 ],
       [-1.0125813 ,  2.1939633 ],
       [ 0.74981135, -0.10980676],
       [-2.3244004 ,  3.0861692 ],
       [-2.3703735 ,  3.2096646 ],
       [ 2.4523177 , -2.129935  ],
       [-2.390077  ,  3.098765  ],
       [ 1.4624766 , -0.89176434],
       [ 2.415533  , -2.1179934 ],
       [-2.3799906 ,  3.044531  ],
       [-2.3766313 ,  2.9987235 ],
       [-2.1951108 ,  2.9488156 ],
       [-2.4254668 ,  2.980787  ],
       [-2.4844823 ,  3.1769319 ],
       [ 2.9107084 , -2.4567668 ],
       [ 2.3056142 , -2.1497815 ],
       [-1.9648348 ,  2.8956995 ],
       [ 2.6260757 , -2.361945  ],
       [-2.4863863 ,

In [ ]:

# 예측 결과 분석 및 평가
print("예측 결과 분석 중...")

# logits에서 클래스 예측값으로 변환
predicted_labels = np.argmax(predictions.logits, axis=1)
prediction_probs = tf.nn.softmax(predictions.logits, axis=1).numpy()

# 실제 레이블 변환 (부정=0, 긍정=1)  
true_labels = comment_valid['label'].apply(lambda x: 0 if x == '부정' else 1).values

# 성능 지표 계산
accuracy = accuracy_score(true_labels, predicted_labels)
print(f'정확도: {accuracy:.4f}')

# 상세한 분류 보고서 출력
print('\n분류 보고서:')
report = classification_report(true_labels, predicted_labels, target_names=['부정', '긍정'])
print(report)

# 혼동 행렬 시각화
cm = confusion_matrix(true_labels, predicted_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['부정', '긍정'], 
            yticklabels=['부정', '긍정'])
plt.title('혼동 행렬 (Confusion Matrix)')
plt.xlabel('예측 레이블')
plt.ylabel('실제 레이블')
plt.show()

# 예측 결과를 DataFrame으로 저장
results_df = pd.DataFrame({
    '텍스트': comment_valid['data'],
    '실제 레이블': comment_valid['label'],
    '예측 레이블': ['부정' if label == 0 else '긍정' for label in predicted_labels],
    '부정 확률': prediction_probs[:, 0],
    '긍정 확률': prediction_probs[:, 1],
    '예측 신뢰도': np.max(prediction_probs, axis=1)
})

print("\n예측 결과 샘플:")
print(results_df[['텍스트', '실제 레이블', '예측 레이블', '예측 신뢰도']].head())

정확도: 0.9604

분류 보고서:
              precision    recall  f1-score   support

          부정       0.97      0.93      0.95        42
          긍정       0.95      0.98      0.97        59

    accuracy                           0.96       101
   macro avg       0.96      0.96      0.96       101
weighted avg       0.96      0.96      0.96       101


예측 결과 샘플:
                                                 텍스트 실제 레이블 예측 레이블
0  유익한 정보 감사합니다. 취미로 바다에 자주 나가는 편인데 이런 기술이 있는지 몰랐네요.     긍정     긍정
1                          발빠른 영입 수고 했네요 고객들이 좋아합니다.     긍정     긍정
2            프로그램C 주제가 우리의 기술력이 바탕이 된다는점~~..항상 응원합니다     긍정     긍정
3  언제나 밝은 미소로 열심히 근무하시는 청원 경찰 분들께 감사드립니다. 항상 건강하시...     긍정     긍정
4               안전사고에 대한 개념도 함께 소개 되었으면 더 완벽했을 것 같네요     부정     부정


In [ ]:
print(results_df.head())

                                                 텍스트 실제 레이블 예측 레이블
0  유익한 정보 감사합니다. 취미로 바다에 자주 나가는 편인데 이런 기술이 있는지 몰랐네요.     긍정     긍정
1                          발빠른 영입 수고 했네요 고객들이 좋아합니다.     긍정     긍정
2            프로그램C 주제가 우리의 기술력이 바탕이 된다는점~~..항상 응원합니다     긍정     긍정
3  언제나 밝은 미소로 열심히 근무하시는 청원 경찰 분들께 감사드립니다. 항상 건강하시...     긍정     긍정
4               안전사고에 대한 개념도 함께 소개 되었으면 더 완벽했을 것 같네요     부정     부정


In [ ]:
# 잘못 예측된 샘플 분석
incorrect_predictions = results_df[results_df['실제 레이블'] != results_df['예측 레이블']].copy()
print(f"\n잘못 예측된 샘플 수: {len(incorrect_predictions)}")

if len(incorrect_predictions) > 0:
    print("\n잘못 예측된 샘플들:")
    print(incorrect_predictions[['텍스트', '실제 레이블', '예측 레이블', '예측 신뢰도']].head(10))
    
    # 신뢰도가 낮은 예측들 확인
    low_confidence = results_df[results_df['예측 신뢰도'] < 0.8].copy()
    print(f"\n신뢰도가 낮은 예측 수 (< 80%): {len(low_confidence)}")
    
    if len(low_confidence) > 0:
        print("신뢰도가 낮은 예측 샘플들:")
        print(low_confidence[['텍스트', '실제 레이블', '예측 레이블', '예측 신뢰도']].head())

print("\n" + "="*50)
print("모델 성능 요약:")
print(f"- 총 테스트 샘플: {len(results_df)}")
print(f"- 정확한 예측: {len(results_df) - len(incorrect_predictions)}")
print(f"- 잘못된 예측: {len(incorrect_predictions)}")
print(f"- 정확도: {accuracy:.4f}")
print(f"- 평균 예측 신뢰도: {results_df['예측 신뢰도'].mean():.4f}")
print("="*50)

In [ ]:
# 모델 및 결과 저장
import os
from datetime import datetime

# 저장 디렉토리 생성
save_dir = "model_output"
os.makedirs(save_dir, exist_ok=True)

# 현재 시간을 포함한 파일명 생성
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 모델 저장 (권장하는 SavedModel 형식)
model_path = os.path.join(save_dir, f"sentiment_bert_model_{timestamp}")
try:
    model.save(model_path)
    print(f"모델이 저장되었습니다: {model_path}")
except Exception as e:
    print(f"모델 저장 중 오류 발생: {e}")

# 결과 CSV 저장
results_path = os.path.join(save_dir, f"prediction_results_{timestamp}.csv")
try:
    results_df.to_csv(results_path, index=False, encoding='utf-8-sig')
    print(f"예측 결과가 저장되었습니다: {results_path}")
except Exception as e:
    print(f"결과 저장 중 오류 발생: {e}")

# 모델 설정 정보 저장
config_info = {
    'model_name': BERT_MODEL_NAME,
    'max_length': MAX_LENGTH,
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'learning_rate': LEARNING_RATE,
    'test_size': TEST_SIZE,
    'final_accuracy': accuracy,
    'timestamp': timestamp
}

config_path = os.path.join(save_dir, f"model_config_{timestamp}.txt")
try:
    with open(config_path, 'w', encoding='utf-8') as f:
        for key, value in config_info.items():
            f.write(f"{key}: {value}\n")
    print(f"모델 설정 정보가 저장되었습니다: {config_path}")
except Exception as e:
    print(f"설정 저장 중 오류 발생: {e}")

print(f"\n모든 결과가 '{save_dir}' 디렉토리에 저장되었습니다.")
results_df = pd.DataFrame({
    '텍스트': comment_test['data'],
    '실제 레이블': comment_test['label'],
    '예측 레이블': ['부정' if label == 0 else '긍정' for label in predicted_labels]
})

# 실제 레이블과 예측 레이블이 다른 경우만 필터링
incorrect_predictions = results_df[results_df['실제 레이블'] != results_df['예측 레이블']]

incorrect_predictions

,텍스트,실제 레이블,예측 레이블
32,요즘같이 우울한 시기에는 희망찬 주제로 부탁해요,부정,긍정
53,다양한 주제로 기획해주세요,부정,긍정
58,주위에 항상 있으나 있는듯 없는듯...중요한지도 고마운지도 모르고 지나치는 산소같은...,긍정,부정
71,인터뷰가 대본을 눈으로 보고 읽고있는 것 같아요,부정,긍정
